In [1]:
# Project idea -> add workout counter in look studio google
# Packages required for this project:
import requests
import pandas as pd
import numpy as np
import pygsheets
import datetime
from datetime import date
import json
import webbrowser
from time import sleep
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import sys
sys.path.append(r'C:\\Users\\Manuel Elizaldi\\Desktop\\Learning-Testing\\PyStrava')
from Functions import *

In [8]:
# Since this notebook is outside of the main folder we have to use . notation
from Credentials.StravaCredentials import data
webbrowser.open(f"https://www.strava.com/oauth/authorize?client_id={data['client_id']}&response_type=code&redirect_uri=http://localhost/&approval_prompt=force&scope=profile:read_all,activity:read_all")
data['code'] = input("From the web broswer enter the code:")

In [9]:
# Getting refresh and access token
token = requests.post(url= 'https://www.strava.com/api/v3/oauth/token',data=data).json()

In [10]:
# Accessing the token json to get refresh token and access token
refresh_token = token['refresh_token']
access_token = token['access_token']

In [11]:
# This API request gives us the general list of activities. 
# The table lacks certain details that we will get from another API request 
activities = retrieve_activities(access_token)

Getting page number: 1
Extraction of page 1 complete
Getting page number: 2
Extraction of page 2 complete
Getting page number: 3
Extraction of page 3 complete
Getting page number: 4
Extraction of page 4 complete
Getting page number: 5
Extraction of page 5 complete
Getting page number: 6
Extraction of page 6 complete
Extraction done


In [13]:
general_table = activities.copy()
general_table = general_table.rename(columns={'elapsed_time':'workout_time_sec'})

In [26]:
general_table.loc[general_table['sport_type'] == 'Run'].head(20)

,id,name,distance,workout_time_sec,total_elevation_gain,sport_type,start_date,achievement_count,athlete_count,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate,average_cadence,elev_high,elev_low
33,10343078755,Morning Run,3014.0,1373,32.0,Run,2023-12-07T13:56:12Z,1,1,"[30.400434834882617, -97.68411417491734]","[30.401210244745016, -97.68475388176739]",2.313,3.978,23.0,131.2,152.0,86.4,209.8,197.4
37,10322687345,Morning Run,21021.1,7400,89.0,Run,2023-12-03T13:10:45Z,4,17,"[29.42491273395717, -98.49457967095077]","[29.421691903844476, -98.49987577646971]",2.842,7.301,18.0,171.9,193.0,89.6,239.4,192.0
39,10309277702,Evening Run,2017.3,939,12.0,Run,2023-12-01T02:03:53Z,0,1,"[27.478308510035276, -99.48788749985397]","[27.47903681360185, -99.48860440403223]",2.174,3.187,25.0,133.5,148.0,84.6,121.8,107.8
41,10298502876,Evening Run,4013.8,1514,13.0,Run,2023-11-29T00:47:37Z,0,1,[],[],2.676,7.312,20.0,152.6,170.0,88.3,124.4,110.4
42,10292826019,Evening Run,6010.4,2713,13.0,Run,2023-11-28T00:34:06Z,0,1,[],[],2.269,4.476,18.0,146.4,167.0,85.8,123.2,112.4
45,10257445914,Morning Run,4855.2,1920,40.0,Run,2023-11-21T15:40:21Z,4,2,"[30.401161713525653, -97.68484256230295]","[30.401187110692263, -97.68477676436305]",2.533,4.532,22.0,146.3,169.0,86.1,213.4,199.2
46,10251037068,Morning Run,6012.5,2765,37.0,Run,2023-11-20T12:48:19Z,0,1,"[30.401362124830484, -97.68470694310963]","[30.401071440428495, -97.68476494587958]",2.175,3.707,28.0,139.5,159.0,85.2,208.2,194.6
48,10233971826,Morning Run,4007.3,1526,7.0,Run,2023-11-17T13:06:00Z,6,1,[],[],2.640,6.324,20.0,150.6,163.0,89.0,158.8,152.0
50,10230744464,Afternoon Run,6002.1,2673,43.0,Run,2023-11-16T19:59:10Z,2,1,"[30.40105945430696, -97.68461658619344]","[30.401184344664216, -97.6845237147063]",2.310,5.860,23.0,142.9,164.0,86.6,214.0,199.4
52,10220071235,Evening Run,3230.6,1225,11.0,Run,2023-11-15T00:07:48Z,0,2,"[31.45808751694858, -100.43619582429528]","[31.458123726770282, -100.43555938638747]",2.637,4.966,21.0,148.8,158.0,85.5,559.0,553.4
